In [1]:
!pip install trl  accelerate git+https://github.com/huggingface/peft.git -Uqqq
!pip install einops wandb -Uqqq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
chex 0.1.82 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.2 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which is incompatible.
dask-cudf 23.8.0 requires dask==2023.7.1, but you have dask 2

In [2]:
!pip install transformers==4.38.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 72.4 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 91.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.2 MB/s eta 0:00:00
  Attempting uninstall: safetensors
    Found existing installation: safetensors 0.3.3
    Uninstalling safetensors-0.3.3:
      Successfully uninstalled safetensors-0.3.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.0
    Uninstalling transformers-4.33.0:
      Successfully uninstalled transformers-4.33.0


In [3]:
!pip show transformers

Name: transformers
Version: 4.38.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /opt/conda/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, trl


In [4]:
!pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 12.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 35.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 11.0.0
    Uninstalling pyarrow-11.0.0:
      Successfully uninstalled pyarrow-11.0.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 require

In [5]:
!pip install -i https://pypi.org/simple/ bitsandbytes  #newline 

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.3 MB/s eta 0:00:0000:0100:01


In [6]:
!pip install accelerate  

In [7]:
import torch
import glob
import pandas as pd
import numpy as np
import re
from peft import get_peft_model, PeftConfig, PeftModel, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from trl import SFTTrainer
from datasets import Dataset

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [28]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_quant_type="nf4",bnb_4bit_use_double_quant=False, bnb_4bit_compute_dtype=torch.bfloat16)


In [29]:


model_name = "Salesforce/codegen-350M-mono"


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config, 
    device_map="auto",  
    trust_remote_code=True,
    
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True) 
#tokenizer.pad_token = tokenizer.eos_token  # modification

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Salesforce--codegen-350M-mono/snapshots/40b7a3b6e99e73bdb497a14b740e7167b3413c74/config.json
Model config CodeGenConfig {
  "_name_or_path": "Salesforce/codegen-350M-mono",
  "activation_function": "gelu_new",
  "architectures": [
    "CodeGenForCausalLM"
  ],
  "attn_pdrop": 0.0,
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "codegen",
  "n_ctx": 2048,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 20,
  "n_positions": 2048,
  "resid_pdrop": 0.0,
  "rotary_dim": 32,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50,


In [12]:
#model_name = "facebook/opt-350m"
#tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True) 


In [27]:
tokenizer.model_max_length

1000000000000000019884624838656

In [30]:
if tokenizer.model_max_length > 100_000:
  tokenizer.model_max_length = 2048

In [12]:
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

In [14]:
DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

In [18]:
from datasets import load_dataset
#dataset = load_dataset("daily_dialog")
#dataset = dataset.remove_columns(["act", "emotion"])
dataset = load_dataset("iamtarun/python_code_instructions_18k_alpaca")
dataset

Generating train split:   0%|          | 0/18612 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt'],
        num_rows: 18612
    })
})

In [19]:
def format_instruction(data):
    processed_data = []
    for sample in data:
        
        data = f"""Instruction:
        Use the Task below and the Input given to write the Response, which is a programming code that can solve the following Task:

        ### Task:
        {sample['instruction']}

        ### Input:
        {sample['input']}

        ### Response:
        {sample['output']}
        """
        processed_data.append(data)
    return pd.DataFrame({"inputs": processed_data})


In [33]:
#train_ds = preprocess_function(dataset['train'], tokenize=False, add_generation_prompt=False)
#train_ds = preprocess_function_chatalpaca(dataset['train'], tokenize=False, add_generation_prompt=False)
train_ds = format_instruction(dataset['train'])
print(train_ds)
train_ds = train_ds.sample(frac = 1)
print(train_ds)
test_ds = train_ds[18001:]
train_ds = train_ds[0:18001]
test_ds

                                                  inputs
0      Instruction:\n        Use the Task below and t...
1      Instruction:\n        Use the Task below and t...
2      Instruction:\n        Use the Task below and t...
3      Instruction:\n        Use the Task below and t...
4      Instruction:\n        Use the Task below and t...
...                                                  ...
18607  Instruction:\n        Use the Task below and t...
18608  Instruction:\n        Use the Task below and t...
18609  Instruction:\n        Use the Task below and t...
18610  Instruction:\n        Use the Task below and t...
18611  Instruction:\n        Use the Task below and t...

[18612 rows x 1 columns]
                                                  inputs
8546   Instruction:\n        Use the Task below and t...
5250   Instruction:\n        Use the Task below and t...
6315   Instruction:\n        Use the Task below and t...
16096  Instruction:\n        Use the Task below and t...
10643

,inputs
11470,Instruction:\n Use the Task below and t...
15254,Instruction:\n Use the Task below and t...
10040,Instruction:\n Use the Task below and t...
4858,Instruction:\n Use the Task below and t...
8580,Instruction:\n Use the Task below and t...
...,...
11284,Instruction:\n Use the Task below and t...
11964,Instruction:\n Use the Task below and t...
5390,Instruction:\n Use the Task below and t...
860,Instruction:\n Use the Task below and t...


In [34]:
train_ds

,inputs
8546,Instruction:\n Use the Task below and t...
5250,Instruction:\n Use the Task below and t...
6315,Instruction:\n Use the Task below and t...
16096,Instruction:\n Use the Task below and t...
10643,Instruction:\n Use the Task below and t...
...,...
15781,Instruction:\n Use the Task below and t...
4673,Instruction:\n Use the Task below and t...
12173,Instruction:\n Use the Task below and t...
6996,Instruction:\n Use the Task below and t...


In [35]:
test_ds

,inputs
11470,Instruction:\n Use the Task below and t...
15254,Instruction:\n Use the Task below and t...
10040,Instruction:\n Use the Task below and t...
4858,Instruction:\n Use the Task below and t...
8580,Instruction:\n Use the Task below and t...
...,...
11284,Instruction:\n Use the Task below and t...
11964,Instruction:\n Use the Task below and t...
5390,Instruction:\n Use the Task below and t...
860,Instruction:\n Use the Task below and t...


In [24]:
#val_ds = preprocess_function(dataset['validation'], tokenize=False, add_generation_prompt=False)
#val_ds
#frames = [train_ds, val_ds]
#train_ds = pd.concat(frames)
#train_ds

In [21]:
print(train_ds['inputs'][0])

Instruction:
        Use the Task below and the Input given to write the Response, which is a programming code that can solve the following Task:

        ### Task:
        Create a function to calculate the sum of a sequence of integers.

        ### Input:
        [1, 2, 3, 4, 5]

        ### Response:
        # Python code
def sum_sequence(sequence):
  sum = 0
  for num in sequence:
    sum += num
  return sum
        


In [11]:
train_ds = pd.read_csv("/kaggle/input/tinycoder/train_dataset_python.csv")

In [12]:
from datasets import DatasetDict


tdf = train_ds
#vdf = df_val
#testdf = test_ds
tds = Dataset.from_pandas(train_ds)
#vds = Dataset.from_pandas(vdf)
#testdf = Dataset.from_pandas(test_ds)


ds = DatasetDict()

ds['train'] = tds
#ds['validation'] = vds
#ds['test'] = testdf
print(ds)

DatasetDict({
    train: Dataset({
        features: ['inputs'],
        num_rows: 18001
    })
})


In [ ]:
#model, tokenizer = setup_chat_format(model, tokenizer, resize_to_multiple_of = 64)

In [ ]:
#special_tokens_dict = {'additional_special_tokens': ['<|user|>', "<|assistant|>"]}
#num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
#print('We have added', num_added_toks, 'tokens')

In [19]:
tokenizer.all_special_tokens

['<|endoftext|>']

In [20]:
tokenizer.model_max_length

2048

In [ ]:
train_dataset = ds["train"]

In [50]:
# Setting arguments for low-rank adaptation 

model = prepare_model_for_kbit_training(model)

lora_alpha = 64 
lora_dropout = 0.01 
lora_rank = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    #target_modules=[ "k_proj", "v_proj", "q_proj"],
    r=lora_rank,
    bias="none",  # setting to 'none' for only training weight params instead of biases
    task_type="CAUSAL_LM")

peft_model = get_peft_model(model, peft_config)

In [51]:
model_name 

'Salesforce/codegen-350M-mono'

In [52]:
# Setting training arguments 
output_dir = "SSahas/codegen_e7"
per_device_train_batch_size = 1
gradient_accumulation_steps = 1  
optim = "paged_adamw_32bit" 
#optim = "adamw_hf"
num_train_epochs = 7
save_strategy="epoch" 
evaluation_strategy = 'steps'
#save_steps="epoch"
#save_steps = 350 
#logging_steps = 200  
logging_strategy = 'steps'
learning_rate = 5e-4
#max_grad_norm = 0.3 # Sets limit for gradient clipping
#max_steps = 1500     # Number of training steps
#warmup_ratio = 0.03 # Portion of steps used for learning_rate to warmup from 0
lr_scheduler_type = "cosine" # I chose cosine to avoid learning plateaus

training_arguments = TrainingArguments(
    #bf16 = True, 
    fp16 = True,
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=num_train_epochs,
    log_level="info",
    save_strategy = save_strategy,
    logging_strategy = logging_strategy,
    #evaluation_strategy = evaluation_strategy,
    #save_steps=save_steps,
    #logging_steps=logging_steps,
    learning_rate=learning_rate,
    #max_grad_norm=max_grad_norm,
    #max_steps=max_steps,
    #warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=False,
    report_to='none'
)

PyTorch: setting up devices


In [54]:

trainer = SFTTrainer(
    model=peft_model,
    #model_init_kwargs=model_kwargs,
    #model_init_kwargs=model_kwargs,
    train_dataset=train_dataset,
    #eval_dataset = eval_dataset,
    peft_config=peft_config,
    max_seq_length=tokenizer.model_max_length,
    dataset_text_field='inputs',
    tokenizer=tokenizer,
    args=training_arguments,
    #neftune_noise_alpha=5
)
#peft_model.config.use_cache = False

Map:   0%|          | 0/18001 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Using auto half precision backend


In [55]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 18,001
  Num Epochs = 7
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 126,007
  Number of trainable parameters = 5,242,880


Step,Training Loss
500,0.620200
1000,0.611000
1500,0.596100
2000,0.580300
2500,0.585700
3000,0.614700
3500,0.609800
4000,0.592000
4500,0.609800
5000,0.592300


Saving model checkpoint to SSahas/codegen_e7/tmp-checkpoint-18001
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Salesforce--codegen-350M-mono/snapshots/40b7a3b6e99e73bdb497a14b740e7167b3413c74/config.json
Model config CodeGenConfig {
  "_name_or_path": "codegen-350M-mono",
  "activation_function": "gelu_new",
  "architectures": [
    "CodeGenForCausalLM"
  ],
  "attn_pdrop": 0.0,
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "codegen",
  "n_ctx": 2048,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 20,
  "n_positions": 2048,
  "resid_pdrop": 0.0,
  "rotary_dim": 32,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generatio